In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import MobileNetV2
from keras.applications.mobilenetv2 import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 35534421
EPOCHS = 150
size = 64
batchsize = 640
lw = 6
channel = 3
STEPS = TOT / EPOCHS / batchsize
NCATS = 340
mixup = 0
center = False
optimizer = 'adam'
cb = 'raw'

In [3]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, 
                               channel=channel, center = center)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

x shape (34000, 64, 64, 3) x max 255
(34000, 64, 64, 3) (34000, 340)
Validation array memory 1.56 GB


In [4]:
K.clear_session()
# model = Xception(input_shape=(size, size, channel), weights=None, classes=NCATS)
model = MobileNetV2(input_shape=(size,size,channel), weights=None, classes=NCATS)

if optimizer == 'sgd':
    opt = SGD(0.002, momentum=0.9, nesterov=True)
    opt = NormalizedOptimizer(opt, normalization='l2')
elif optimizer == 'adam':
    opt = Adam(lr=0.002)

model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 65, 65, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 32, 32, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 32, 32, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [5]:
train_datagen = image_generator_xd(df_path = '../input/train_all_balance.csv', 
                                   size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel, mixup=mixup, center=center)

In [6]:
model_prefix = 'mobile_pointcnt{}_lw{}_balance_{}_{}'.format(size, lw, mixup, optimizer, cb)
if center:
    model_prefix += '_center'
if channel == 3:
    model_prefix += '_parts_limit'
print(model_prefix)
check_path = "./models/{}.model".format(model_prefix)
fold = 9
if cb == 'raw':
    callbks = [
        ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=5,
                          min_delta=0.005, mode='max', cooldown=3, verbose=1),
        callbacks.ModelCheckpoint(check_path,
                                    monitor='val_top_3_accuracy', 
                                    mode = 'max', save_best_only=True, verbose=1),
        callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
        EarlyStopping(monitor='val_top_3_accuracy', mode='max',patience=20, verbose=1)
    ]
elif cb == 'snap':
    snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,
                                       nb_snapshots=1,
                                       size = size,
                                       init_lr=1e-3, 
                                       fold=fold)

    callbks = snapshot.get_callbacks(model_prefix = model_prefix)

mobile_pointcnt64_lw6_balance_0_adam_parts_limit


In [8]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks 
)

Epoch 1/150
371/370 [==============================] - 96s 258ms/step - loss: 3.7375 - categorical_crossentropy: 3.7375 - categorical_accuracy: 0.2122 - top_3_accuracy: 0.3647 - val_loss: 14.2197 - val_categorical_crossentropy: 14.2197 - val_categorical_accuracy: 0.0136 - val_top_3_accuracy: 0.0285

Epoch 00001: val_top_3_accuracy improved from -inf to 0.02853, saving model to ./models/mobile_pointcnt64_lw6_balance_0_adam_parts_limit.model
Epoch 2/150
371/370 [==============================] - 87s 235ms/step - loss: 2.2426 - categorical_crossentropy: 2.2426 - categorical_accuracy: 0.4637 - top_3_accuracy: 0.6770 - val_loss: 13.8239 - val_categorical_crossentropy: 13.8239 - val_categorical_accuracy: 0.0224 - val_top_3_accuracy: 0.0499

Epoch 00002: val_top_3_accuracy improved from 0.02853 to 0.04985, saving model to ./models/mobile_pointcnt64_lw6_balance_0_adam_parts_limit.model
Epoch 3/150
371/370 [==============================] - 87s 235ms/step - loss: 1.8975 - categorical_crossentro

371/370 [==============================] - 88s 238ms/step - loss: 1.1422 - categorical_crossentropy: 1.1422 - categorical_accuracy: 0.7104 - top_3_accuracy: 0.8718 - val_loss: 3.6222 - val_categorical_crossentropy: 3.6222 - val_categorical_accuracy: 0.3206 - val_top_3_accuracy: 0.5144

Epoch 00020: val_top_3_accuracy did not improve from 0.76271
Epoch 21/150
371/370 [==============================] - 88s 238ms/step - loss: 1.1349 - categorical_crossentropy: 1.1349 - categorical_accuracy: 0.7129 - top_3_accuracy: 0.8727 - val_loss: 4.5502 - val_categorical_crossentropy: 4.5502 - val_categorical_accuracy: 0.2393 - val_top_3_accuracy: 0.4014

Epoch 00021: val_top_3_accuracy did not improve from 0.76271
Epoch 22/150
371/370 [==============================] - 88s 238ms/step - loss: 1.1267 - categorical_crossentropy: 1.1267 - categorical_accuracy: 0.7145 - top_3_accuracy: 0.8747 - val_loss: 1.9167 - val_categorical_crossentropy: 1.9167 - val_categorical_accuracy: 0.5648 - val_top_3_accuracy:

371/370 [==============================] - 88s 237ms/step - loss: 1.0230 - categorical_crossentropy: 1.0230 - categorical_accuracy: 0.7385 - top_3_accuracy: 0.8895 - val_loss: 1.2598 - val_categorical_crossentropy: 1.2598 - val_categorical_accuracy: 0.6889 - val_top_3_accuracy: 0.8580

Epoch 00041: val_top_3_accuracy improved from 0.85694 to 0.85803, saving model to ./models/mobile_pointcnt64_lw6_balance_0_adam_parts_limit.model
Epoch 42/150
371/370 [==============================] - 88s 237ms/step - loss: 1.0183 - categorical_crossentropy: 1.0183 - categorical_accuracy: 0.7400 - top_3_accuracy: 0.8895 - val_loss: 1.1975 - val_categorical_crossentropy: 1.1975 - val_categorical_accuracy: 0.7071 - val_top_3_accuracy: 0.8701

Epoch 00042: val_top_3_accuracy improved from 0.85803 to 0.87015, saving model to ./models/mobile_pointcnt64_lw6_balance_0_adam_parts_limit.model
Epoch 43/150
371/370 [==============================] - 88s 237ms/step - loss: 1.0235 - categorical_crossentropy: 1.0235 


Epoch 00061: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00061: val_top_3_accuracy improved from 0.90341 to 0.90447, saving model to ./models/mobile_pointcnt64_lw6_balance_0_adam_parts_limit.model
Epoch 62/150
371/370 [==============================] - 88s 237ms/step - loss: 0.9056 - categorical_crossentropy: 0.9056 - categorical_accuracy: 0.7690 - top_3_accuracy: 0.9051 - val_loss: 0.8931 - val_categorical_crossentropy: 0.8931 - val_categorical_accuracy: 0.7729 - val_top_3_accuracy: 0.9064

Epoch 00062: val_top_3_accuracy improved from 0.90447 to 0.90641, saving model to ./models/mobile_pointcnt64_lw6_balance_0_adam_parts_limit.model
Epoch 63/150
371/370 [==============================] - 88s 238ms/step - loss: 0.9014 - categorical_crossentropy: 0.9014 - categorical_accuracy: 0.7693 - top_3_accuracy: 0.9065 - val_loss: 0.8934 - val_categorical_crossentropy: 0.8934 - val_categorical_accuracy: 0.7730 - val_top_3_accuracy: 0.9064

Epoch 00063: val_top_3_acc

371/370 [==============================] - 88s 238ms/step - loss: 0.8875 - categorical_crossentropy: 0.8875 - categorical_accuracy: 0.7731 - top_3_accuracy: 0.9086 - val_loss: 0.8705 - val_categorical_crossentropy: 0.8705 - val_categorical_accuracy: 0.7781 - val_top_3_accuracy: 0.9096

Epoch 00082: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 00082: val_top_3_accuracy did not improve from 0.90962
Epoch 83/150
371/370 [==============================] - 88s 237ms/step - loss: 0.8793 - categorical_crossentropy: 0.8793 - categorical_accuracy: 0.7745 - top_3_accuracy: 0.9089 - val_loss: 0.8702 - val_categorical_crossentropy: 0.8702 - val_categorical_accuracy: 0.7779 - val_top_3_accuracy: 0.9095

Epoch 00083: val_top_3_accuracy did not improve from 0.90962
Epoch 84/150
371/370 [==============================] - 88s 237ms/step - loss: 0.8812 - categorical_crossentropy: 0.8812 - categorical_accuracy: 0.7751 - top_3_accuracy: 0.9091 - val_loss: 0.8696 - val_categor

371/370 [==============================] - 88s 238ms/step - loss: 0.8809 - categorical_crossentropy: 0.8809 - categorical_accuracy: 0.7743 - top_3_accuracy: 0.9090 - val_loss: 0.8666 - val_categorical_crossentropy: 0.8666 - val_categorical_accuracy: 0.7797 - val_top_3_accuracy: 0.9102

Epoch 00103: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.

Epoch 00103: val_top_3_accuracy improved from 0.91018 to 0.91021, saving model to ./models/mobile_pointcnt64_lw6_balance_0_adam_parts_limit.model
Epoch 104/150
371/370 [==============================] - 88s 237ms/step - loss: 0.8774 - categorical_crossentropy: 0.8774 - categorical_accuracy: 0.7744 - top_3_accuracy: 0.9091 - val_loss: 0.8666 - val_categorical_crossentropy: 0.8666 - val_categorical_accuracy: 0.7796 - val_top_3_accuracy: 0.9101

Epoch 00104: val_top_3_accuracy did not improve from 0.91021
Epoch 105/150
371/370 [==============================] - 88s 238ms/step - loss: 0.8834 - categorical_crossentropy: 0.8834 -

In [9]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input, 
                             channel=channel, center = center)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}


x shape (112199, 64, 64, 3) x max 255
(112199, 3) (112199, 64, 64, 3)
Test array memory 5.14 GB


In [10]:
def doodle_predict(model, model_path, x_test):
    model.load_weights(model_path)

    test_predictions = model.predict(x_test, batch_size=128, verbose=1)
    top3 = preds2catids(test_predictions)
    top3cats = top3.replace(id2cat)
    test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
    submission = test[['key_id', 'word']]

    import kaggle_util
    kaggle_util.save_result(submission,  
                            '../result/{}.csv'.format(model_prefix), 
                            'quickdraw-doodle-recognition', 
                            send=True, index=False)

In [11]:
doodle_predict(model, check_path, x_test)

112199/112199 [==============================] - 15s 135us/step
save result
upload result
cmd: kaggle competitions submit -c quickdraw-doodle-recognition -f ../result/mobile_pointcnt64_lw6_balance_0_adam_parts_limit.csv.7z -m "submit"
